In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 7.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=ab03dd7673b3f82109dfb30fb13d000c7319915c63f2a2c1dedb674883d1a96f
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
import yfinance as yf

data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "NVAX",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "60d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "30m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2020-12-23 09:30:00-05:00,111.879997,113.832199,107.081001,113.150002,1040295
2020-12-23 10:00:00-05:00,113.160004,119.230003,112.809998,117.333504,712430
2020-12-23 10:30:00-05:00,117.266403,119.580002,117.152298,118.570000,392045
2020-12-23 11:00:00-05:00,118.377800,118.389999,116.260101,116.349998,198065
2020-12-23 11:30:00-05:00,116.269997,116.940002,116.029999,116.389999,95438
...,...,...,...,...,...
2021-03-22 13:30:00-04:00,236.413605,237.979996,236.413605,237.500000,52596
2021-03-22 14:00:00-04:00,237.395004,238.179993,236.270004,237.669998,62813
2021-03-22 14:30:00-04:00,237.839996,238.787201,236.580002,238.787201,55748


In [5]:
prices = data.reset_index()
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype                           
---  ------    --------------  -----                           
 0   Datetime  775 non-null    datetime64[ns, America/New_York]
 1   Open      775 non-null    float64                         
 2   High      775 non-null    float64                         
 3   Low       775 non-null    float64                         
 4   Close     775 non-null    float64                         
 5   Volume    775 non-null    int64                           
dtypes: datetime64[ns, America/New_York](1), float64(4), int64(1)
memory usage: 36.5 KB


In [6]:
prices['Datetime'] = prices['Datetime'].astype(str)

In [7]:
#remove -5:00
prices['Datetime'] = prices['Datetime'].str.replace('-05:00',"")
prices['Datetime'] = prices['Datetime'].str.replace('-04:00',"")

In [8]:
prices = prices[(prices['Datetime']>= '2020-12-22 09:30:00') & (prices['Datetime']<='2021-03-22 15:30:00')]
prices

,Datetime,Open,High,Low,Close,Volume
0,2020-12-23 09:30:00,111.879997,113.832199,107.081001,113.150002,1040295
1,2020-12-23 10:00:00,113.160004,119.230003,112.809998,117.333504,712430
2,2020-12-23 10:30:00,117.266403,119.580002,117.152298,118.570000,392045
3,2020-12-23 11:00:00,118.377800,118.389999,116.260101,116.349998,198065
4,2020-12-23 11:30:00,116.269997,116.940002,116.029999,116.389999,95438
...,...,...,...,...,...,...
770,2021-03-22 13:30:00,236.413605,237.979996,236.413605,237.500000,52596
771,2021-03-22 14:00:00,237.395004,238.179993,236.270004,237.669998,62813
772,2021-03-22 14:30:00,237.839996,238.787201,236.580002,238.787201,55748
773,2021-03-22 15:00:00,238.783493,241.479996,237.660004,238.084000,160555


In [9]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 0 to 774
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  775 non-null    object 
 1   Open      775 non-null    float64
 2   High      775 non-null    float64
 3   Low       775 non-null    float64
 4   Close     775 non-null    float64
 5   Volume    775 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 42.4+ KB


In [11]:
sentiment = pd.read_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/NVAX/60days_tweets.csv',lineterminator='\n')
sentiment.head()

,Datetime,Text,processed_tweet
0,2021-01-21 00:27:43+00:00,$NVAX look like it’s finna boom according to t...,nvax look like its finna boom according to the...
1,2021-01-21 00:31:02+00:00,@AkremAk @InovioPharma man a company that does...,man a company that doesnt have debt and even h...
2,2021-01-21 01:27:00+00:00,@NateSilver538 @thehowie Agreed. Ultimately it...,agreed ultimately it all sort of comes togethe...
3,2021-01-21 01:38:50+00:00,@AlexDelarge6553 @MentalHealthVR @Bill01234567...,for all those paying close attention to rlftf ...
4,2021-01-21 02:34:34+00:00,@RNAiAnalyst Slaoui has always protected the b...,slaoui has always protected the big guns hes b...


In [12]:
sentiment = sentiment.rename(columns={'Datetime': 'created_at'})

In [13]:
#remove +00:00
sentiment['created_at'] = sentiment['created_at'].str.slice(0,19)

In [14]:
sentiment.head()

,created_at,Text,processed_tweet
0,2021-01-21 00:27:43,$NVAX look like it’s finna boom according to t...,nvax look like its finna boom according to the...
1,2021-01-21 00:31:02,@AkremAk @InovioPharma man a company that does...,man a company that doesnt have debt and even h...
2,2021-01-21 01:27:00,@NateSilver538 @thehowie Agreed. Ultimately it...,agreed ultimately it all sort of comes togethe...
3,2021-01-21 01:38:50,@AlexDelarge6553 @MentalHealthVR @Bill01234567...,for all those paying close attention to rlftf ...
4,2021-01-21 02:34:34,@RNAiAnalyst Slaoui has always protected the b...,slaoui has always protected the big guns hes b...


In [15]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13510 entries, 0 to 13509
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   created_at       13510 non-null  object
 1   Text             13510 non-null  object
 2   processed_tweet  13510 non-null  object
dtypes: object(3)
memory usage: 316.8+ KB


In [16]:
# convert datetime and created at to same datetime object
prices['Datetime'] = pd.to_datetime(prices['Datetime'])
sentiment['created_at'] = pd.to_datetime(sentiment['created_at'])

In [17]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 0 to 774
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  775 non-null    datetime64[ns]
 1   Open      775 non-null    float64       
 2   High      775 non-null    float64       
 3   Low       775 non-null    float64       
 4   Close     775 non-null    float64       
 5   Volume    775 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 42.4 KB


In [18]:
prices

,Datetime,Open,High,Low,Close,Volume
0,2020-12-23 09:30:00,111.879997,113.832199,107.081001,113.150002,1040295
1,2020-12-23 10:00:00,113.160004,119.230003,112.809998,117.333504,712430
2,2020-12-23 10:30:00,117.266403,119.580002,117.152298,118.570000,392045
3,2020-12-23 11:00:00,118.377800,118.389999,116.260101,116.349998,198065
4,2020-12-23 11:30:00,116.269997,116.940002,116.029999,116.389999,95438
...,...,...,...,...,...,...
770,2021-03-22 13:30:00,236.413605,237.979996,236.413605,237.500000,52596
771,2021-03-22 14:00:00,237.395004,238.179993,236.270004,237.669998,62813
772,2021-03-22 14:30:00,237.839996,238.787201,236.580002,238.787201,55748
773,2021-03-22 15:00:00,238.783493,241.479996,237.660004,238.084000,160555


In [19]:
# only close column is needed
prices = prices.drop(columns=['Open','High','Low','Volume'])

In [20]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13510 entries, 0 to 13509
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   created_at       13510 non-null  datetime64[ns]
 1   Text             13510 non-null  object        
 2   processed_tweet  13510 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 316.8+ KB


In [22]:
sentiment = sentiment.rename(columns={'created_at': 'Datetime'})
sentiment

,Datetime,Text,processed_tweet
0,2021-01-21 00:27:43,$NVAX look like it’s finna boom according to t...,nvax look like its finna boom according to the...
1,2021-01-21 00:31:02,@AkremAk @InovioPharma man a company that does...,man a company that doesnt have debt and even h...
2,2021-01-21 01:27:00,@NateSilver538 @thehowie Agreed. Ultimately it...,agreed ultimately it all sort of comes togethe...
3,2021-01-21 01:38:50,@AlexDelarge6553 @MentalHealthVR @Bill01234567...,for all those paying close attention to rlftf ...
4,2021-01-21 02:34:34,@RNAiAnalyst Slaoui has always protected the b...,slaoui has always protected the big guns hes b...
...,...,...,...
13505,2021-03-22 22:58:52,$NVAX went from 4$ to 300$ under 2 years (84x)...,nvax went from to under years x and some peopl...
13506,2021-03-22 23:26:37,Join Robinhood with my link and we'll both get...,join robinhood with my link and well both get ...
13507,2021-03-22 23:28:13,What Happened Today in the US Stock Market?\n\...,what happened today in the us stock market chw...
13508,2021-03-22 23:45:57,$TGTX $ACIU $MSFT $AAPL $ZM $ZS $AMD $M $TSLA ...,tgtx aciu msft aapl zm zs amd m tsla crsp nflx...


In [23]:
# drop na of sentiment
sentiment = sentiment.dropna()

In [24]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13510 entries, 0 to 13509
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         13510 non-null  datetime64[ns]
 1   Text             13510 non-null  object        
 2   processed_tweet  13510 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 422.2+ KB


In [25]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 0 to 774
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  775 non-null    datetime64[ns]
 1   Close     775 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 18.2 KB


In [26]:
df_merge = pd.merge_asof(sentiment,prices,left_on='Datetime',right_on="Datetime",direction='backward')
df_merge

,Datetime,Text,processed_tweet,Close
0,2021-01-21 00:27:43,$NVAX look like it’s finna boom according to t...,nvax look like its finna boom according to the...,124.870003
1,2021-01-21 00:31:02,@AkremAk @InovioPharma man a company that does...,man a company that doesnt have debt and even h...,124.870003
2,2021-01-21 01:27:00,@NateSilver538 @thehowie Agreed. Ultimately it...,agreed ultimately it all sort of comes togethe...,124.870003
3,2021-01-21 01:38:50,@AlexDelarge6553 @MentalHealthVR @Bill01234567...,for all those paying close attention to rlftf ...,124.870003
4,2021-01-21 02:34:34,@RNAiAnalyst Slaoui has always protected the b...,slaoui has always protected the big guns hes b...,124.870003
...,...,...,...,...
13505,2021-03-22 22:58:52,$NVAX went from 4$ to 300$ under 2 years (84x)...,nvax went from to under years x and some peopl...,237.679993
13506,2021-03-22 23:26:37,Join Robinhood with my link and we'll both get...,join robinhood with my link and well both get ...,237.679993
13507,2021-03-22 23:28:13,What Happened Today in the US Stock Market?\n\...,what happened today in the us stock market chw...,237.679993
13508,2021-03-22 23:45:57,$TGTX $ACIU $MSFT $AAPL $ZM $ZS $AMD $M $TSLA ...,tgtx aciu msft aapl zm zs amd m tsla crsp nflx...,237.679993


In [27]:
df_merge1 = df_merge[df_merge.Close.notnull()]
df_merge1

,Datetime,Text,processed_tweet,Close
0,2021-01-21 00:27:43,$NVAX look like it’s finna boom according to t...,nvax look like its finna boom according to the...,124.870003
1,2021-01-21 00:31:02,@AkremAk @InovioPharma man a company that does...,man a company that doesnt have debt and even h...,124.870003
2,2021-01-21 01:27:00,@NateSilver538 @thehowie Agreed. Ultimately it...,agreed ultimately it all sort of comes togethe...,124.870003
3,2021-01-21 01:38:50,@AlexDelarge6553 @MentalHealthVR @Bill01234567...,for all those paying close attention to rlftf ...,124.870003
4,2021-01-21 02:34:34,@RNAiAnalyst Slaoui has always protected the b...,slaoui has always protected the big guns hes b...,124.870003
...,...,...,...,...
13505,2021-03-22 22:58:52,$NVAX went from 4$ to 300$ under 2 years (84x)...,nvax went from to under years x and some peopl...,237.679993
13506,2021-03-22 23:26:37,Join Robinhood with my link and we'll both get...,join robinhood with my link and well both get ...,237.679993
13507,2021-03-22 23:28:13,What Happened Today in the US Stock Market?\n\...,what happened today in the us stock market chw...,237.679993
13508,2021-03-22 23:45:57,$TGTX $ACIU $MSFT $AAPL $ZM $ZS $AMD $M $TSLA ...,tgtx aciu msft aapl zm zs amd m tsla crsp nflx...,237.679993


In [28]:
import datetime

df_final = pd.DataFrame()

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(2021, 1, 21), datetime.datetime(2021, 3, 23))
listofdates = []
for i in range (len(total_days)):
  date_time = total_days[i] + datetime.timedelta(minutes=30)
  date_time = date_time + datetime.timedelta(hours=15)
  end_time = date_time.strftime("%Y-%m-%d %H:%M:%S")

  date_time2 = total_days[i] + datetime.timedelta(hours=9)
  start_time = date_time2.strftime("%Y-%m-%d %H:%M:%S")
  
  listofdates.append([end_time, start_time])

# print(len(total_days))

for i in range(len(total_days)):
  df_final = df_final.append(df_merge1[(df_merge1['Datetime']<=listofdates[i][0]) & (df_merge1['Datetime']>=listofdates[i][1])])

In [29]:
df_final

,Datetime,Text,processed_tweet,Close
17,2021-01-21 09:39:25,SK Chemicals share price soaring in South Kore...,sk chemicals share price soaring in south kore...,125.140099
18,2021-01-21 10:53:06,@NickytaLeb @KurtadikarKetan @DeItaone https:/...,contracts concluded w aznm doses sny gskm dose...,127.050003
19,2021-01-21 11:20:23,I wish I got more of $SRPT $FB $BTC $ARK $NVAX...,i wish i got more of srpt fb btc ark nvax dis ...,129.000000
20,2021-01-21 11:50:21,$NVAX\nResults from UK &amp; SA before End of ...,nvax results from uk amp sa before end of mont...,127.809998
21,2021-01-21 11:55:02,$NVAX $VXRT NEW ARTICLE : Better Coronavirus S...,nvax vxrt new article better coronavirus stoc...,127.809998
...,...,...,...,...
13425,2021-03-22 15:10:01,"$NVAX trying to breakout above 235, 242-245, 2...",nvax trying to breakout above amp are levels a...,238.084000
13426,2021-03-22 15:10:12,$NVAX the news about to come! 🚀🔥,nvax the news about to come,238.084000
13427,2021-03-22 15:10:41,$nvax $245 next target https://t.co/mA2AHeNu57,nvax next target,238.084000
13428,2021-03-22 15:16:19,@matthewherper @Novavax again you choose misle...,again you choose misleading data from nvax in ...,238.084000


In [30]:
df_final = df_final.reset_index(drop=True)
df_final

,Datetime,Text,processed_tweet,Close
0,2021-01-21 09:39:25,SK Chemicals share price soaring in South Kore...,sk chemicals share price soaring in south kore...,125.140099
1,2021-01-21 10:53:06,@NickytaLeb @KurtadikarKetan @DeItaone https:/...,contracts concluded w aznm doses sny gskm dose...,127.050003
2,2021-01-21 11:20:23,I wish I got more of $SRPT $FB $BTC $ARK $NVAX...,i wish i got more of srpt fb btc ark nvax dis ...,129.000000
3,2021-01-21 11:50:21,$NVAX\nResults from UK &amp; SA before End of ...,nvax results from uk amp sa before end of mont...,127.809998
4,2021-01-21 11:55:02,$NVAX $VXRT NEW ARTICLE : Better Coronavirus S...,nvax vxrt new article better coronavirus stoc...,127.809998
...,...,...,...,...
3694,2021-03-22 15:10:01,"$NVAX trying to breakout above 235, 242-245, 2...",nvax trying to breakout above amp are levels a...,238.084000
3695,2021-03-22 15:10:12,$NVAX the news about to come! 🚀🔥,nvax the news about to come,238.084000
3696,2021-03-22 15:10:41,$nvax $245 next target https://t.co/mA2AHeNu57,nvax next target,238.084000
3697,2021-03-22 15:16:19,@matthewherper @Novavax again you choose misle...,again you choose misleading data from nvax in ...,238.084000


In [31]:
df_final["Comp"] = ''
df_final["Negative"] = ''
df_final["Neutral"] = ''
df_final["Positive"] = ''
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2021-01-21 09:39:25,SK Chemicals share price soaring in South Kore...,sk chemicals share price soaring in south kore...,125.140099,,,,
1,2021-01-21 10:53:06,@NickytaLeb @KurtadikarKetan @DeItaone https:/...,contracts concluded w aznm doses sny gskm dose...,127.050003,,,,
2,2021-01-21 11:20:23,I wish I got more of $SRPT $FB $BTC $ARK $NVAX...,i wish i got more of srpt fb btc ark nvax dis ...,129.000000,,,,
3,2021-01-21 11:50:21,$NVAX\nResults from UK &amp; SA before End of ...,nvax results from uk amp sa before end of mont...,127.809998,,,,
4,2021-01-21 11:55:02,$NVAX $VXRT NEW ARTICLE : Better Coronavirus S...,nvax vxrt new article better coronavirus stoc...,127.809998,,,,
...,...,...,...,...,...,...,...,...
3694,2021-03-22 15:10:01,"$NVAX trying to breakout above 235, 242-245, 2...",nvax trying to breakout above amp are levels a...,238.084000,,,,
3695,2021-03-22 15:10:12,$NVAX the news about to come! 🚀🔥,nvax the news about to come,238.084000,,,,
3696,2021-03-22 15:10:41,$nvax $245 next target https://t.co/mA2AHeNu57,nvax next target,238.084000,,,,
3697,2021-03-22 15:16:19,@matthewherper @Novavax again you choose misle...,again you choose misleading data from nvax in ...,238.084000,,,,


In [32]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3699 entries, 0 to 3698
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         3699 non-null   datetime64[ns]
 1   Text             3699 non-null   object        
 2   processed_tweet  3699 non-null   object        
 3   Close            3699 non-null   float64       
 4   Comp             3699 non-null   object        
 5   Negative         3699 non-null   object        
 6   Neutral          3699 non-null   object        
 7   Positive         3699 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 231.3+ KB


In [33]:
df_final['processed_tweet'] = df_final['processed_tweet'].astype(str)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3699 entries, 0 to 3698
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         3699 non-null   datetime64[ns]
 1   Text             3699 non-null   object        
 2   processed_tweet  3699 non-null   object        
 3   Close            3699 non-null   float64       
 4   Comp             3699 non-null   object        
 5   Negative         3699 non-null   object        
 6   Neutral          3699 non-null   object        
 7   Positive         3699 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 231.3+ KB


In [34]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
df_final['Comp'] = [analyzer.polarity_scores(x)['compound'] for x in df_final['processed_tweet']]
df_final['Negative'] = [analyzer.polarity_scores(x)['neg'] for x in df_final['processed_tweet']]
df_final['Neutral'] = [analyzer.polarity_scores(x)['neu'] for x in df_final['processed_tweet']]
df_final['Positive'] = [analyzer.polarity_scores(x)['pos'] for x in df_final['processed_tweet']]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [35]:
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2021-01-21 09:39:25,SK Chemicals share price soaring in South Kore...,sk chemicals share price soaring in south kore...,125.140099,0.2960,0.000,0.937,0.062
1,2021-01-21 10:53:06,@NickytaLeb @KurtadikarKetan @DeItaone https:/...,contracts concluded w aznm doses sny gskm dose...,127.050003,0.0000,0.000,1.000,0.000
2,2021-01-21 11:20:23,I wish I got more of $SRPT $FB $BTC $ARK $NVAX...,i wish i got more of srpt fb btc ark nvax dis ...,129.000000,0.4019,0.000,0.828,0.172
3,2021-01-21 11:50:21,$NVAX\nResults from UK &amp; SA before End of ...,nvax results from uk amp sa before end of mont...,127.809998,0.0000,0.000,1.000,0.000
4,2021-01-21 11:55:02,$NVAX $VXRT NEW ARTICLE : Better Coronavirus S...,nvax vxrt new article better coronavirus stoc...,127.809998,0.4404,0.000,0.854,0.146
...,...,...,...,...,...,...,...,...
3694,2021-03-22 15:10:01,"$NVAX trying to breakout above 235, 242-245, 2...",nvax trying to breakout above amp are levels a...,238.084000,0.0000,0.000,1.000,0.000
3695,2021-03-22 15:10:12,$NVAX the news about to come! 🚀🔥,nvax the news about to come,238.084000,0.0000,0.000,1.000,0.000
3696,2021-03-22 15:10:41,$nvax $245 next target https://t.co/mA2AHeNu57,nvax next target,238.084000,0.0000,0.000,1.000,0.000
3697,2021-03-22 15:16:19,@matthewherper @Novavax again you choose misle...,again you choose misleading data from nvax in ...,238.084000,0.1027,0.224,0.564,0.212


In [36]:
df_final = df_final.groupby(df_final.Datetime.dt.floor('30min')).mean()

In [37]:
df_final = df_final.drop(columns=['Negative','Neutral','Positive'])
df_final

,Close,Comp
Datetime,,
2021-01-21 09:30:00,125.140099,0.296000
2021-01-21 10:30:00,127.050003,0.000000
2021-01-21 11:00:00,129.000000,0.401900
2021-01-21 11:30:00,127.809998,0.214233
2021-01-21 12:00:00,126.500000,0.000000
...,...,...
2021-03-22 13:00:00,236.160004,0.042700
2021-03-22 13:30:00,237.500000,0.200080
2021-03-22 14:00:00,237.669998,0.530700


In [38]:
df_final.to_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/NVAX/final.csv')